In [ ]:
!pip install pandas_ta
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=1c591e62fb7ceeabf6fb7378032a4622b96973b51c35ef3f8dfca441f56bf87e
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 38.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [ ]:
import yfinance as yf
import pandas as pd

In [ ]:
import yfinance as yf
import pandas_ta as ta


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def merge_dataframes(df1, df2):
    # Calculate the difference and percent change for the first dataframe
    df1['Close_Change'] = df1['Close'].diff()
    df1['Close_Percent_Change'] = df1['Close'].pct_change() * 100

    # Save the old column names of the second dataframe
    old_columns = list(df2.columns)

    # Rename the columns of the second dataframe
    df2.columns = ["num", "date", "Question", "Username", "Link", "SubjectParagraph"]

    # Create a new DataFrame with the old columns as the first row
    new_df = pd.DataFrame([old_columns], columns=df2.columns)

    # Concatenate the new DataFrame with the second DataFrame
    df2 = pd.concat([new_df, df2], ignore_index=True)

    # Convert the 'date' column to datetime format and set it as index for the second dataframe
    df2['date'] = pd.to_datetime(df2['date'], format='%Y-%m-%d')
    df2.set_index('date', inplace=True)

    common_dates =df1.index.intersection(df2.index)
    df1_common = df1.loc[common_dates]
    df2_common = df2.loc[common_dates]

    # Merge the two dataframes based on the date index using outer join
    result_df = df1_common.merge(df2_common, left_index=True, right_index=True, how='outer')

    return result_df

In [ ]:

# Downloading data for AAPL for the year 2020
dataAAPL = yf.download('AAPL', start='2010-01-01', end='2020-12-31')
dataMSFT = yf.download('MSFT', start='2010-01-01', end='2020-12-31' )
dataNVDA = yf.download('NVDA', start='2010-01-01', end='2020-12-31' )
dataTSLA = yf.download('TSLA', start='2010-01-01', end='2020-12-31' )

# Calculating the daily close change and percentage change
# dataAAPL['Close_Change'] = data['Close'].diff()
# dataAAPL['Close_Percent_Change'] = data['Close'].pct_change() * 100
applestock = pd.read_csv('/content/drive/MyDrive/data/stockapple.csv')
teslastock = pd.read_csv('/content/drive/MyDrive/data/stocktesla.csv')
nvidiastock = pd.read_csv('/content/drive/MyDrive/data/stocknvidia.csv')
microsoftstock = pd.read_csv('/content/drive/MyDrive/data/stockmicrosoft.csv')

resultapple = merge_dataframes(dataAAPL,applestock)
resulttesla = merge_dataframes(dataTSLA,teslastock)
resultnvidia = merge_dataframes(dataNVDA,nvidiastock)
resultmicrosoft = merge_dataframes(dataMSFT,microsoftstock)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
common_dates =resultapple.index.intersection(resultmicrosoft.index)
common_dates

DatetimeIndex(['2011-07-08', '2012-03-30', '2013-03-14', '2013-07-12',
               '2013-09-16', '2014-05-22', '2014-10-20', '2014-10-30',
               '2014-11-10', '2014-12-02',
               ...
               '2020-12-15', '2020-12-17', '2020-12-18', '2020-12-21',
               '2020-12-22', '2020-12-23', '2020-12-24', '2020-12-28',
               '2020-12-29', '2020-12-30'],
              dtype='datetime64[ns]', length=426, freq=None)

In [ ]:
result_df = pd.concat([resultapple, resultmicrosoft,resultnvidia,resultmicrosoft], ignore_index=True)

In [ ]:
result_df['Close_Percent_Change'].size

4516

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from torch.nn import Linear
from torch.nn import MSELoss  # or use CrossEntropyLoss for classification
from transformers import AutoConfig
from tqdm import tqdm

# Configuration
model_name = 'bert-base-uncased'
num_labels = 1  # Adjust based on your task, e.g., 2 for binary classification
learning_rate = 2e-5
batch_size = 16
num_epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
model = BertForSequenceClassification.from_pretrained(model_name, config=config)

# Modify the classifier
model.classifier = Linear(model.config.hidden_size, num_labels)
torch.nn.init.kaiming_normal_(model.classifier.weight, nonlinearity='relu')
torch.nn.init.zeros_(model.classifier.bias)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameter containing:
tensor([0.], requires_grad=True)

In [ ]:
texts = result_df['SubjectParagraph'].tolist()  # Ensure this list doesn't contain NaN or None values
labels = result_df['Close_Percent_Change'].tolist()

In [ ]:
# Tokenize text data
inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
inputs = {key: value.to(device) for key, value in inputs.items()}

labels = torch.tensor(labels, dtype=torch.float)  # Use torch.float for regression tasks
labels = labels.to(device)  # Move labels to the specified device (CPU or GPU)


In [ ]:
# Prepare DataLoader
dataset = torch.utils.data.TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = MSELoss()  # Or CrossEntropyLoss for classification

In [ ]:
# Training loop with tqdm
model.to(device)
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        b_input_ids, b_attention_mask, b_labels = batch
        model.zero_grad()
        outputs = model(b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Average Training Loss: {avg_train_loss}")

# Save the model
model_path = '/content/drive/MyDrive/Modelsavetransformers/model_save4'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print(f"Model saved to {model_path}")

Training Epoch 1: 100%|██████████| 226/226 [1:30:13<00:00, 23.96s/it]


Epoch 1, Average Training Loss: 7.699818543628254


Training Epoch 2: 100%|██████████| 226/226 [1:31:23<00:00, 24.26s/it]


Epoch 2, Average Training Loss: 7.397625438407459


Training Epoch 3: 100%|██████████| 226/226 [1:28:42<00:00, 23.55s/it]


Epoch 3, Average Training Loss: 5.8974247258321375
Model saved to /content/drive/MyDrive/Modelsavetransformers/model_save4


In [ ]:
# Detailed evaluation
model.eval()
total_eval_loss = 0
total_eval_accuracy = 0

for batch in tqdm(val_loader, desc="Evaluating"):
    b_input_ids, b_attention_mask, b_labels = batch
    with torch.no_grad():
        outputs = model(b_input_ids, attention_mask=b_attention_mask, labels=b_labels)

    loss = outputs.loss
    total_eval_loss += loss.item()

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    total_eval_accuracy += (predictions == b_labels).sum().item()

avg_val_loss = total_eval_loss / len(val_loader)
avg_val_accuracy = total_eval_accuracy / len(val_dataset)

print(f"Validation Loss: {avg_val_loss}")
print(f"Validation Accuracy: {avg_val_accuracy}")

Evaluating: 100%|██████████| 57/57 [06:27<00:00,  6.80s/it]

Validation Loss: 8.0983642904382
Validation Accuracy: 0.00331858407079646
